In [148]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit import QuantumCircuit,transpile, assemble
from qiskit.visualization import plot_histogram
from qiskit.quantum_info.operators import Operator

from qiskit_aer import Aer
import numpy as np
import random

In [149]:
def ChangeBasis(first_basis, second_basis):
    first_eigenvalues, first_eigenvectors = np.linalg.eig(first_basis)
    second_eigenvalues, second_eigenvectors = np.linalg.eig(second_basis)
    U = np.outer(second_eigenvectors[0], first_eigenvectors[0]) + np.outer(second_eigenvectors[1], first_eigenvectors[1])
    return U

In [234]:
def getStateVector(qc):
    qc.save_statevector()
    aer_simulator = Aer.get_backend('aer_simulator')
    
    compiled_circuit = transpile(qc, aer_simulator)
    result = aer_simulator.run(compiled_circuit, shots=1).result()
    
    statevector = result.get_statevector(compiled_circuit)
    print("Statevector:", statevector)

In [235]:
def measure(qc):
        
    backend = Aer.get_backend('qasm_simulator')
    transpiled_qc = transpile(qc, backend)
    qobj = assemble(transpiled_qc, shots=1)
    result = backend.run(qc, shots=1).result()
    
    counts = result.get_counts()
    measured_bit = counts.most_frequent()
    return measured_bit

In [236]:
def GenerateBellState():
    qc = QuantumCircuit(3, 3)
    qc.h(0)
    qc.cx(0, 1)
    qc.cx(1, 2)
    return qc

In [244]:
def calculate(sequence_leangh, qc, basis):
    Alice_basis = random.choices([0, 1, 2], [1/3, 1/3, 1/3], k=sequence_leangh)
    Bob_basis = random.choices([1, 2, 3], [1/3, 1/3, 1/3], k=sequence_leangh)
    measured_bits = []
    for i in range(sequence_leangh):
        U = ChangeBasis([[1, 0],[0, -1]], basis[Alice_basis[i]])
        U = Operator(U)
        qc[i].unitary(U, 0)

        U = ChangeBasis([[1, 0],[0, -1]], basis[Bob_basis[i]])
        U = Operator(U)
        qc[i].unitary(U, 1)
        
        qc[i].measure(0, 0)
        qc[i].measure(1, 1)
        qc[i].measure(2, 2)

        outcome = measure(qc[i])
        #print(Alice_basis[i], Bob_basis[i], outcome)
        measured_bits.append(outcome)
        

    return Alice_basis, Bob_basis, measured_bits
        

In [238]:
def correlation(measured_bits):
    return sum([1 if a == b else -1 for a, b in measured_bits]) / len(measured_bits)


In [239]:
def BellTest(sequeance_leangh, Alice_basis, Bob_basis, measured_bits):
    
    settings = {
        'ab': [], 'ab_prime': [], 'a_prime_b': [], 'a_prime_b_prime': []
    }
    
    for i in range(sequeance_leangh):
        if Alice_basis[i] == 0 and Bob_basis[i] == 1:
            settings['ab'].append(i)
        elif Alice_basis[i] == 0 and Bob_basis[i] == 3:
            settings['ab_prime'].append(i)
        elif Alice_basis[i] == 2 and Bob_basis[i] == 1:
            settings['a_prime_b'].append(i)
        elif Alice_basis[i] == 2 and Bob_basis[i] == 3:
            settings['a_prime_b_prime'].append(i)

    E_ab = correlation(
        [measured_bits[i][0:2] for i in settings['ab']])
    E_ab_prime = correlation(
        [measured_bits[i][0:2] for i in settings['ab_prime']])
    E_a_prime_b = correlation(
        [measured_bits[i][0:2] for i in settings['a_prime_b']])
    E_a_prime_b_prime = correlation(
        [measured_bits[i][0:2] for i in settings['a_prime_b_prime']])
    print(E_ab, E_ab_prime, E_a_prime_b, E_a_prime_b_prime)

    S = abs(E_ab - E_ab_prime + E_a_prime_b + E_a_prime_b_prime)
    return S
            

In [242]:
def E91():
    sequence_leangh = 500
    qc = [GenerateBellState() for i in range(sequence_leangh)]
    X = np.array([[0, 1], [1, 0]])
    Z = np.array([[1, 0], [0, -1]])
    theta = 0
    basis = []
    basis.append(np.cos(theta) * X + np.sin(theta) * Z)
    basis.append(np.cos(theta + np.pi/4) * X + np.sin(theta + np.pi/4) * Z)
    basis.append(np.cos(theta + np.pi/2) * X + np.sin(theta + np.pi/2) * Z)
    basis.append(np.cos(theta + 3*np.pi/4) * X + np.sin(theta + 3* np.pi/4) * Z)

    Alice_basis, Bob_basis, measured_bits = calculate(sequence_leangh, qc, basis)
    
    S = BellTest(sequence_leangh, Alice_basis, Bob_basis, measured_bits)
    print(S)


In [246]:
E91()

0.7142857142857143 0.6862745098039216 0.8367346938775511 0.75
1.614745898359344


In [98]:
X = np.array([[0, 1], [1, 0]])
Z = np.array([[1, 0], [0, -1]])
theta = 0
first_basis = np.cos(theta) * X + np.sin(theta) * Z
second_basis = np.cos(theta + np.pi/4) * X + np.sin(theta + np.pi/4) * Z
third_basis = np.cos(theta + np.pi/2) * X + np.sin(theta + np.pi/4) * Z
ChangeBasis(Z, for)

array([[1., 0.],
       [0., 1.]])

In [160]:
qc = QuantumCircuit(3, 3)
qc.h(0)
qc.cx(0, 1)
qc.cx(0, 2)
getStateVector(qc)

Statevector: Statevector([0.70710678+0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.        +0.j, 0.        +0.j,
             0.        +0.j, 0.70710678+0.j],
            dims=(2, 2, 2))
